In [ ]:
import os
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
from zipfile import ZipFile
import matplotlib.image as mimage

%matplotlib inline

import keras
import tensorflow as tf
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.losses import binary_crossentropy
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications import VGG16

# Loading Data

Here the aim is to take the shape info as input and give skeleton as output

In [ ]:
img_datagen = ImageDataGenerator(rescale=1/255.0 , validation_split=0.1)


seed = 10
train_image_generator =img_datagen.flow_from_directory('../input/datasetiiitd/images_out',batch_size=10,classes=['images_vgg'],color_mode="rgb",target_size=(224, 224),
                                                    class_mode=None,seed = seed , shuffle = False , subset = 'training')

train_skeleton_generator = img_datagen.flow_from_directory('../input/datasetiiitd/skels_out',batch_size=10,color_mode = 'grayscale',classes=['skels_vgg'],target_size=(224, 224),
                                                   class_mode=None,seed = seed , shuffle = False , subset = 'training')

val_image_generator =img_datagen.flow_from_directory('../input/datasetiiitd/images_out',batch_size=10,classes=['images_vgg'],color_mode="rgb",target_size=(224, 224),
                                                    class_mode=None,seed = seed , shuffle = False , subset = 'validation')

val_skeleton_generator = img_datagen.flow_from_directory('../input/datasetiiitd/skels_out',batch_size=10,color_mode = 'grayscale',classes=['skels_vgg'],target_size=(224, 224),
                                                   class_mode=None,seed = seed , shuffle = False , subset = 'validation')


#shapeinfo_generator = img_datagen.flow_from_directory('../input/datasetiiitd/shapeinfo_out',batch_size=10,color_mode = 'grayscale',classes=['shape_info'],target_size=(224, 224),
#                                                   class_mode=None,seed = seed , shuffle = False)


train_generator = zip( train_image_generator , train_skeleton_generator)
val_generator = zip(val_image_generator , val_skeleton_generator)

In [ ]:
plt.imshow(val_skeleton_generator[0][0])


# HELPER FUNCTIONS

In [ ]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

## Model 1


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
def model1(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained VGG16 Model """
    vgg16 = VGG16(include_top=False, weights="imagenet", input_tensor=inputs )
    
    vgg16.trainable = False

    """ Encoder """
    s1 = vgg16.get_layer("block1_conv2").output ## (512 x 512)
    s2 = vgg16.get_layer("block2_conv2").output ## (256 x 256)
    s3 = vgg16.get_layer("block3_conv3").output ## (128 x 128)
    s4 = vgg16.get_layer("block4_conv3").output ## (64 x 64)

    """ Bridge """
    b1 = vgg16.get_layer("block5_conv3").output ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512) ## (64 x 64)
    d2 = decoder_block(d1, s3, 256) ## (128 x 128)
    d3 = decoder_block(d2, s2, 128) ## (256 x 256)
    d4 = decoder_block(d3, s1, 64) ## (512 x 512)

    """ Output """
    output1 = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    output2 = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs = [output1 , output2], name="VGG16_U-Net")
    return model

model1 = model1((224,224,3))
model1.summary()

## Freezing model1

In [ ]:
model1.trainable = False

In [ ]:
model1.summary()

In [ ]:
model1 = load_model('../input/model1-trained/45_epochs.h5')

In [ ]:
model1.trainable = False
model1.summary()

## Defining model2

In [ ]:
def model2(input_shape):
    inputs = Input(input_shape , name = "input_3")
    
    
    """ Encoder """
    b1_c1 = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu", name ="block1_conv1")(inputs)
    b1_c2 = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu" , name = "block1_conv2" )(b1_c1)
    b1_p = MaxPool2D(pool_size=(2,2),strides=(2,2) , name = "block1_pool")(b1_c2)
    
    b2_c1 = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu", name = "block2_conv1")(b1_p)
    b2_c2 = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu", name = "block2_conv2")(b2_c1)
    b2_p = MaxPool2D(pool_size=(2,2),strides=(2,2) , name = "block2_pool")(b2_c2)
    
    b3_c1 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", name = "block3_conv1")(b2_p)
    b3_c2 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", name = "block3_conv2")(b3_c1)
    b3_c3 = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", name = "block3_conv3")(b3_c2)
    b3_p = MaxPool2D(pool_size=(2,2),strides=(2,2) , name = "block3_pool")(b3_c3)
    
    b4_c1 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" ,name = "block4_conv1" )(b3_p)
    b4_c2 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" ,name = "block4_conv2" )(b4_c1)
    b4_c3 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" ,name = "block4_conv3" )(b4_c2)
    b4_p = MaxPool2D(pool_size=(2,2),strides=(2,2) ,name = "block4_pool")(b4_c3)
    
    b5_c1 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , name = "block5_conv1")(b4_p)
    b5_c2 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , name = "block5_conv2")(b5_c1)
    
    
    """ Bridge """
    b5_c3 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu" , name = "block5_conv3")(b5_c2)
    
    
    """ Decoder """
    s1 = b1_c2
    s2 = b2_c2
    s3 = b3_c3
    s4 = b4_c3
    
    d1 = decoder_block(b5_c3, s4, 512)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)  
    
    output1 = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    
    model = Model(inputs, outputs = output1, name="VGG16_U-Net2")
    return model

In [ ]:
model2 = model2((224,224,5))
#model2.summary()

In [ ]:
## Taking pretrained vgg for getting weights
inputs = Input((224,224,3))
vgg16 = VGG16(include_top=False, weights="imagenet", input_tensor=inputs  )
weights_list_vgg16 = vgg16.get_weights()
weights_list_model = model2.get_weights()
layers_fixed = ['block1_conv2' , 'block1_pool', 'block2_conv1' ,'block2_conv2' , 'block2_pool',
               'block3_conv1' , 'block3_conv2' , 'block3_conv3' , 'block3_pool' , 'block4_conv1' ,
               'block4_conv2' , 'block4_conv3' , 'block4_pool' , 'block5_conv1' , 'block5_conv2' , 'block5_conv3']
for layer_name in layers_fixed:
    
    #print(layer_name)
    weights = vgg16.get_layer(layer_name).get_weights()
    model2.get_layer(layer_name).set_weights(weights)
    model2.get_layer(layer_name).trainable = False

In [ ]:
model2.summary()

In [ ]:
def main_model():
    
    input_rgb = Input((224,224,3))
    input_colordistance = model1(input_rgb)[0]
    input_spatialdistance = model1(input_rgb)[1]
    
    model2_input = concatenate([input_rgb, input_colordistance , input_spatialdistance],axis=-1)
    
    model2_output = model2(model2_input)
    
    main = Model(inputs = [input_rgb] , outputs = [model2_output] )

    return main

In [ ]:
final_model = main_model()

In [ ]:
final_model.summary()

In [ ]:
final_model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=['AUC'])
callbacks = [
    EarlyStopping(patience=3, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000001, verbose=1 , min_delta=0.0001),
    ModelCheckpoint('model-tgs-salt.h5', verbose=1, save_best_only=True)
]

In [ ]:
results = final_model.fit_generator(train_generator,steps_per_epoch=887,epochs=150,callbacks=callbacks,
                                   validation_data=val_generator,
                                   validation_steps=88)

In [ ]:
final_model = load_model('../input/final-model/model-tgs-salt.h5')

In [ ]:
i = 5
sample_inp2 = val_image_generator[0][i]
sample_inp2 = np.expand_dims(sample_inp2,0)
pred = final_model.predict(sample_inp2)
plt.subplot(131)
plt.imshow(pred[0] > 0.1, cmap='gray')
plt.grid(b=None)
plt.subplot(132)
plt.imshow(val_skeleton_generator[0][i] > 0.1 , cmap='gray')
plt.grid(b=None)
plt.subplot(133)
plt.imshow(val_image_generator[0][i])
plt.grid(b=None)

In [ ]:
final_model.summary()

In [ ]:
BFScore_perimage(train_skeleton_generator[0][5] , (pred[0]>0.15).astype("int32") ,tolerance=2)

In [ ]:
BFScore_batch(skeleton_generator[2] , (final_model.predict(image_generator[2]) > 0.2).astype("int32") ,tolerance=2)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support,f1_score

# np.unique((pred[0] >0.1).astype("int32"))
# np.unique(skeleton_generator[0][5])
#f1_score(skeleton_generator[0][5] , (pred[0]>0.1).astype("int32"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image

from skimage import io
import glob

def calc_precision_recall(im1 , im2 , tolerance = 2):
    
    
    im1_white_co_ordinates = np.argwhere(im1 == 1)
    im2_white_co_ordinates = np.argwhere(im2 == 1)
    hits = []
    
    for i in im2_white_co_ordinates:
        
        d = np.square( im1_white_co_ordinates[: , 0] - i[0])   + \
            np.square( im1_white_co_ordinates[: , 1] - i[1]) 
        
        hits.append(np.any( d < tolerance*tolerance))
        
    count = np.sum(hits)
    
    try:
        precision_recall = count / im2_white_co_ordinates.shape[0]
        
    except ZeroDivisionError:
        precision_recall = 0
        
    
    return precision_recall , count , im2_white_co_ordinates.shape[0]


def BFScore_perimage( img_predicted , img_gt , tolerance):
    
    precision , p_numerator , p_denominator = calc_precision_recall(img_gt , img_predicted , tolerance  ) 
    recall , r_numerator , r_denominator = calc_precision_recall(img_predicted , img_gt , tolerance )
    
    f1 = 0
    
    try:
        f1 = 2*recall*precision/(recall + precision)    # F1 score 
    except:  
        f1 = 0        
    return f1


def BFScore_batch( imgs_predicted  ,  imgs_gt , tolerance = 2 , threshold = 0.10):
    
    
    """
    imgs_predicted will have shape (BATCH_SIZE , 224,224,1)
    imgs_gt will have shape (BATCH_SIZE , 224,224,1)
    
    """
    BATCH_SIZE = imgs_predicted.shape[0]
    
    imgs_predicted = (imgs_predicted > threshold).astype(np.int32)
    BF_net = 0
    
    for i in range(BATCH_SIZE):
        
        BF_net += BFScore_perimage( imgs_predicted[i] , imgs_gt[i] , tolerance)
        
        
    return BF_net/BATCH_SIZE